In [1]:
import uproot
import awkward as ak
import numpy as np
import sklearn.metrics as m
import numba as nb
from coffea.nanoevents.methods import vector
import os

import boost_histogram as bh
from matplotlib import pyplot as plt
import matplotlib as mpl
from cycler import cycler
import mplhep as hep
use_helvet = True  ## true: use helvetica for plots, make sure the system have the font installed
if use_helvet:
    CMShelvet = hep.style.CMS
    CMShelvet['font.sans-serif'] = ['Helvetica', 'Arial']
    plt.style.use(CMShelvet)
else:
    plt.style.use(hep.style.CMS)

import os, sys  
sys.path.insert(1, '/data/pku/home/licq/cH/zz_v2/selection/analyzer')

In [3]:
# df0 = uproot.lazy("/home/pku/licq/cH/zz/samples/zzntuples/MC_UL18/GluGluHToZZTo4L_M125_TuneCP5_13TeV_powheg2_JHUGenV7011_pythia8/merged.root")
# df = uproot.lazy("/home/pku/licq/cH/zz/samples/zzntuples/MC_UL18/HPlusCharm_4FS_MuRFScaleDynX0p50_HToZZTo4L_M125_TuneCP5_13TeV_amcatnloFXFX_JHUGenV7011_pythia8/merged.root")
df = uproot.lazy("/home/pku/licq/cH/zz_v2/samples/zzntuples/MC_UL18/HPlusCharm_4FS_MuRFScaleDynX0p50_HToZZTo4L_M125_TuneCP5_13TeV_amcatnlo_JHUGenV7011_pythia8/merged.root")

In [19]:
for b in ak.fields(df):
    if 'var *' in str(ak.type(df[b])) and 'var * var *' not in str(ak.type(df[b])):
        num = ak.num(df[b])
        print(b, num)
        if num[-1] > 10:
            print('alert!!')

qcdWeights [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ... 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
nnloWeights [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ... 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
pdfWeights [110, 110, 110, 110, 110, 110, 110, 110, ... 110, 110, 110, 110, 110, 110, 110, 110]
alert!!
lep_d0BS [0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 4, ... 4, 0, 0, 0, 0, 0, 0, 4, 0, 4, 0, 0, 0]
lep_d0PV [0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 4, ... 4, 0, 0, 0, 0, 0, 0, 4, 0, 4, 0, 0, 0]
lep_numberOfValidPixelHits [0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 4, ... 4, 0, 0, 0, 0, 0, 0, 4, 0, 4, 0, 0, 0]
lep_trackerLayersWithMeasurement [0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 4, ... 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0]
lep_p [0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 4, ... 4, 0, 0, 0, 0, 0, 0, 4, 0, 4, 0, 0, 0]
lep_ecalEnergy [0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 4, ... 4, 0, 0, 0, 0, 0, 0, 4, 0, 4, 0, 0, 0]
lep_isEB [0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 4, ... 4, 0, 4, 0, 0, 0, 0, 4, 0, 4, 

In [13]:
for b in ak.fields(df):
    print(b)

Run
Event
LumiSect
nVtx
nInt
PV_x
PV_y
PV_z
BS_x
BS_y
BS_z
BS_xErr
BS_yErr
BS_zErr
BeamWidth_x
BeamWidth_y
BeamWidth_xErr
BeamWidth_yErr
finalState
triggersPassed
passedTrig
passedFullSelection
passedZ4lSelection
passedQCDcut
passedZ1LSelection
passedZ4lZ1LSelection
passedZ4lZXCRSelection
passedZXCRSelection
nZXCRFailedLeptons
genWeight
k_ggZZ
k_qqZZ_qcd_dPhi
k_qqZZ_qcd_M
k_qqZZ_qcd_Pt
k_qqZZ_ewk
qcdWeights
nnloWeights
pdfWeights
pdfRMSup
pdfRMSdown
pdfENVup
pdfENVdown
pileupWeight
pileupWeightUp
pileupWeightDn
dataMCWeight
eventWeight
prefiringWeight
prefiringWeightECAL
prefiringWeightMuon
crossSection
lep_d0BS
lep_d0PV
lep_numberOfValidPixelHits
lep_trackerLayersWithMeasurement
lep_p
lep_ecalEnergy
lep_isEB
lep_isEE
singleBS_Lep_pt
singleBS_Lep_ptError
singleBS_Lep_eta
singleBS_Lep_phi
singleBS_Lep_mass
singleBS_Lep_d0
vtxLep_BS_pt
vtxLep_BS_pt_NoRoch
vtxLep_BS_ptError
vtxLep_BS_eta
vtxLep_BS_phi
vtxLep_BS_mass
vtxLep_BS_d0
vtxLep_pt
vtxLep_ptError
vtxLep_eta
vtxLep_phi
vtxLep_mass
s

# Produce v1

In [3]:
sample_map = {
    'ggH125': 'GluGluHToZZTo4L_M125_TuneCP5_13TeV_powheg2_JHUGenV7011_pythia8',
    'HC_3FS': 'HPlusCharm_3FS_MuRFScaleDynX0p50_HToZZTo4L_M125_TuneCP5_13TeV_amcatnlo_JHUGenV7011_pythia8',
    'HC_4FS': 'HPlusCharm_4FS_MuRFScaleDynX0p50_HToZZTo4L_M125_TuneCP5_13TeV_amcatnlo_JHUGenV7011_pythia8',
    # 'HC_4FSFxFx': 'HPlusCharm_4FS_MuRFScaleDynX0p50_HToZZTo4L_M125_TuneCP5_13TeV_amcatnloFXFX_JHUGenV7011_pythia8',
}

store_flat_only = False

from importlib import reload
import ntupler
ntupler = reload(ntupler)
for sam in sample_map:
    print(sam)
    in_path = f"/home/pku/licq/cH/zz_v2/samples/zzntuples/MC_UL18/{sample_map[sam]}/merged.root"
    if store_flat_only:
        out_path = f"/home/pku/licq/cH/zz_v2/samples/gen_training_dataset/v1-flat/{sam}.root"
    else:
        out_path = f"/home/pku/licq/cH/zz_v2/samples/gen_training_dataset/v1/{sam}.root"
    
    if not os.path.exists(os.path.dirname(out_path)):
        os.makedirs(os.path.dirname(out_path))
    
    df = uproot.lazy(in_path)
    is_hc = sam.startswith('HC_')
    with uproot.recreate(out_path, compression=uproot.LZ4(4)) as fw:
        fw['Events'] = ntupler.ntupler(df, is_hc=is_hc, store_flat_only=store_flat_only)


ggH125
HC_3FS
HC_4FS


# Produce ntuple with pfcands

In [5]:
sample_map = {
    'ggH125': 'GluGluHToZZTo4L_M125_TuneCP5_13TeV_powheg2_JHUGenV7011_pythia8',
    # 'HC_3FS': 'HPlusCharm_3FS_MuRFScaleDynX0p50_HToZZTo4L_M125_TuneCP5_13TeV_amcatnlo_JHUGenV7011_pythia8',
    'HC_4FS': 'HPlusCharm_4FS_MuRFScaleDynX0p50_HToZZTo4L_M125_TuneCP5_13TeV_amcatnlo_JHUGenV7011_pythia8',
    'HC_4FSFxFx': 'HPlusCharm_4FS_MuRFScaleDynX0p50_HToZZTo4L_M125_TuneCP5_13TeV_amcatnloFXFX_JHUGenV7011_pythia8',
}

store_pfcands = True

from importlib import reload
import ntupler
ntupler = reload(ntupler)
for sam in sample_map:
    print(sam)
    in_path = f"/home/pku/licq/cH/zz_v2/samples/zzntuples/MC_UL18_addPFCands_try3/{sample_map[sam]}/merged.root"
    out_path = f"/home/pku/licq/cH/zz_v2/samples/gen_training_dataset/v2p1-pfcands/{sam}.root"
    
    if not os.path.exists(os.path.dirname(out_path)):
        os.makedirs(os.path.dirname(out_path))
    
    df = uproot.lazy(in_path)
    is_hc = sam.startswith('HC_')
    with uproot.recreate(out_path) as fw:
        fw['Events'] = ntupler.ntupler(df, is_hc=is_hc, store_pfcands=True)


ggH125
HC_4FS
HC_4FSFxFx


# Produce ntuples for r2 dataset (v3)

In [2]:
sample_map = {
    'ggH125': 'GluGluHToZZTo4L_M125_TuneCP5_13TeV_powheg2_JHUGenV7011_pythia8',
    # 'HC_3FS': 'HPlusCharm_3FS_MuRFScaleDynX0p50_HToZZTo4L_M125_TuneCP5_13TeV_amcatnlo_JHUGenV7011_pythia8',
    # 'HC_4FS': 'HPlusCharm_4FS_MuRFScaleDynX0p50_HToZZTo4L_M125_TuneCP5_13TeV_amcatnlo_JHUGenV7011_pythia8',
    'HC_4FSFxFx': 'HPlusCharm_4FS_MuRFScaleDynX0p50_HToZZTo4L_M125_TuneCP5_13TeV_amcatnloFXFX_JHUGenV7011_pythia8',
}

store_pfcands = True

from importlib import reload
import ntupler
ntupler = reload(ntupler)
for sam in sample_map:
    print(sam)
    in_path = f"/home/pku/licq/cH/zz_v2/samples/zzntuples/MC_UL18_r2_main/{sample_map[sam]}/merged.root"
    out_path = f"/home/pku/licq/cH/zz_v2/samples/gen_training_dataset/v3/{sam}.root"
    
    if not os.path.exists(os.path.dirname(out_path)):
        os.makedirs(os.path.dirname(out_path))
    
    df = uproot.lazy(in_path)
    is_hc = sam.startswith('HC_')
    with uproot.recreate(out_path) as fw:
        fw['Events'] = ntupler.ntupler(df, is_hc=is_hc, store_gen=False, store_pfcands=False, store_flat_only=True)


ggH125


PermissionError: [Errno 13] Permission denied: '/home/pku/licq/cH/zz_v2/samples/gen_training_dataset/v3/ggH125.root'

In [4]:
sample_map = {
    'ggH125': 'GluGluHToZZTo4L_M125_TuneCP5_13TeV_powheg2_JHUGenV7011_pythia8',
    # 'HC_3FS': 'HPlusCharm_3FS_MuRFScaleDynX0p50_HToZZTo4L_M125_TuneCP5_13TeV_amcatnlo_JHUGenV7011_pythia8',
    # 'HC_4FS': 'HPlusCharm_4FS_MuRFScaleDynX0p50_HToZZTo4L_M125_TuneCP5_13TeV_amcatnlo_JHUGenV7011_pythia8',
    'HC_4FSFxFx': 'HPlusCharm_4FS_MuRFScaleDynX0p50_HToZZTo4L_M125_TuneCP5_13TeV_amcatnloFXFX_JHUGenV7011_pythia8',
}
in_path = f"/home/pku/licq/cH/zz_v2/samples/zzntuples/MC_UL18_r2_main/{sample_map['ggH125']}/merged.root"
df = uproot.lazy(in_path)

In [11]:
from importlib import reload
import ntupler
ntupler = reload(ntupler)
ntupler.ntupler(df[:10], is_hc=is_hc, store_gen=False, store_pfcands=False, store_flat_only=True)

... ParticleNet_CvsB: -10, px: -15.6, py: -2.05, pz: 6.76, e: 17.4}
... ParticleNet_CvsB: -10, px: -15.6, py: -2.05, pz: 6.76, e: 17.4}
... ParticleNet_CvsB: -10, px: -15.6, py: -2.05, pz: 6.76, e: 17.4}


{'Run': <Array [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] type='10 * uint64'>,
 'Event': <Array [530005, 530004, ... 530015, 530008] type='10 * uint64'>,
 'LumiSect': <Array [531, 531, 531, 531, ... 531, 531, 531] type='10 * uint64'>,
 'genWeight': <Array [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] type='10 * float32'>,
 'mass4l': <Array [-1, 125, -1, -1, ... -1, -1, -1, -1] type='10 * float32'>,
 'D_bkg_kin': <Array [999, 0.784, 999, ... 999, 999, 999] type='10 * float32'>,
 'is_hc': <Array [True, True, True, ... True, True, True] type='10 * bool'>,
 'pass_fiducial': <Array [False, True, False, ... False, False] type='10 * bool'>,
 'H_pt': <Array [-1, 21.6, -1, -1, ... -1, -1, -1, -1] type='10 * float64'>,
 'H_pt_reg': <Array [-1, 3.12, -1, -1, ... -1, -1, -1, -1] type='10 * float64'>,
 'H_eta': <Array [-1, 2.1, -1, -1, ... -1, -1, -1, -1] type='10 * float64'>,
 'H_phi': <Array [-1, 0.306, -1, -1, ... -1, -1, -1, -1] type='10 * float64'>,
 'H_px': <Array [-1, 20.6, -1, -1, ... -1, -1, -1, -1] type='10 * float

In [21]:
a = uproot.lazy("/home/pku/licq/cH/zz_v2/samples/gen_training_dataset/v3/ggH125.root").cleanedjet_leadc2c_pt

In [22]:
ak.min(a[a>0])

15.000015258789062